In [ ]:
import numpy as np
import numba as nb
from scipy import interpolate
from spectral.io import envi
import matplotlib.pyplot as plt

In [ ]:
def convex_hull(wvl, spectrum):
    """Computes the convex hull of a set of 2D points.

    Input: an iterable sequence of (x, y) pairs representing the points.
    Output: a list of vertices of the convex hull in counter-clockwise order,
      starting from the vertex with the lexicographically smallest coordinates.
    Implements the algorithm CONVEXHULL(P) described by  Mark de Berg, Otfried
    Cheong, Marc van Kreveld, and Mark Overmars, in Computational Geometry:
    Algorithm and Applications, pp. 6-7 in Chapter 1

    :param points: A N X 2 matrix with the wavelengths as the first column
    :return: The convex hull vector
    """
    'The starting points be the first two points'
    xcnt, y = wvl[:2], spectrum[:2]
    'Now iterate over the other points'
    for ii in range(2, spectrum.shape[0], 1):
        'check next prospective convex hull members'
        xcnt = np.append(xcnt, wvl[ii])
        y = np.append(y, spectrum[ii])
        flag = True

        while (flag == True):
            'Check if a left turn occurs at the central member'
            a1 = (y[-2] - y[-3]) / (xcnt[-2] - xcnt[-3])
            a2 = (y[-1] - y[-2]) / (xcnt[-1] - xcnt[-2])
            if (a2 > a1):
                xcnt[-2] = xcnt[-1]
                xcnt = xcnt[:-1]
                y[-2] = y[-1]
                y = y[:-1]
                flag = (xcnt.shape[0] > 2);
            else:
                flag = False

    return np.vstack((xcnt, y))

In [ ]:
@nb.jit("f4[:, :](f4[:], f4[:])")
def convex_hull_jit(wvl, spectrum):
    """Computes the convex hull of a set of 2D points.

    Input: an iterable sequence of (x, y) pairs representing the points.
    Output: a list of vertices of the convex hull in counter-clockwise order,
      starting from the vertex with the lexicographically smallest coordinates.
    Implements the algorithm CONVEXHULL(P) described by  Mark de Berg, Otfried
    Cheong, Marc van Kreveld, and Mark Overmars, in Computational Geometry:
    Algorithm and Applications, pp. 6-7 in Chapter 1

    :param points: A N X 2 matrix with the wavelengths as the first column
    :return: The convex hull vector
    """
    'The starting points be the first two points'
    xcnt, y = wvl[:2], spectrum[:2]
    'Now iterate over the other points'
    for ii in range(2, spectrum.shape[0], 1):
        'check next prospective convex hull members'
        xcnt = np.append(xcnt, wvl[ii])
        y = np.append(y, spectrum[ii])
        flag = True

        while (flag == True):
            'Check if a left turn occurs at the central member'
            a1 = (y[-2] - y[-3]) / (xcnt[-2] - xcnt[-3])
            a2 = (y[-1] - y[-2]) / (xcnt[-1] - xcnt[-2])
            if (a2 > a1):
                xcnt[-2] = xcnt[-1]
                xcnt = xcnt[:-1]
                y[-2] = y[-1]
                y = y[:-1]
                flag = (xcnt.shape[0] > 2);
            else:
                flag = False

    return np.vstack((xcnt, y))

In [ ]:
'Get an image'
imgName = '/Volume2/data/CRISM/AMS/packageTrail/FRT00003E12/FRT00003E12_07_IF166L_TRR3_atcr_sabcondv3_1_Lib1112_1_4_5_redAb_MS.img'
imgHdrName = imgName.replace(".img", ".hdr")
img = envi.open(imgHdrName, imgName)
cube = img.load()
[rows, cols, bands] = img.shape
cube = img.load()
cube = np.asarray(cube[:,:,4:244], dtype="f4")

In [ ]:
header = envi.read_envi_header(imgHdrName)
wvl = header['wavelength']
wvl = wvl[4:244]
wvl = np.asarray(wvl, dtype='single')

In [ ]:
row = 309
col = 258
spectrum = np.squeeze(cube[row, col, :])

In [ ]:
plt.figure()
plt.plot(wvl, spectrum)

In [ ]:
%time t1 = convex_hull(wvl, spectrum)
t1 = convex_hull(wvl, spectrum)

In [ ]:
%time t2 = convex_hull_jit(wvl, spectrum)
t2 = convex_hull_jit(wvl, spectrum)

In [ ]:
print t1

In [ ]:
print t2.shape

In [ ]:
f = interpolate.interp1d(np.squeeze(t1[0,:]), np.squeeze(t1[1,:]))
ycnt = f(wvl)

plt.figure()
plt.plot(wvl, spectrum, 'b-')
plt.plot(wvl, ycnt, 'g--')

In [ ]:
%time f = interpolate.interp1d(np.squeeze(t1[0,:]), np.squeeze(t1[1,:]))
%time ycnt = f(wvl)

In [ ]:
4.81*100

In [ ]:
0.391*1000

In [ ]:
0.0725*10000

In [ ]:
((11.5*1000) + 482 +365)/(542+482+365)

In [ ]:
class c1():
    def f1(self, wvl, spectrum):
        t3 = convex_hull_jit(wvl, spectrum)
        return t3
    

In [ ]:
obj = c1()
%time t3 = obj.f1(wvl,spectrum)